# minimal-interface

Notebook using the camera feed and buttons to move the robot.

In [ ]:
# Insérez ici le numéro de votre équipe
# (en) Insert here your team number
VOTRE_NUMERO_EQUIPE = 1
# modifier cette variable selon que vous contrôler la simulation ou le robot réel
# (en) change this variable according to whether you control the simulation or the real robot
SIM = True

In [ ]:
# Import des paquets requis et configuration du ROS Master
# (en) Import of required packages and configuration of the ROS Master
import sys; sys.path.append('/opt/ros/noetic/lib/python3/dist-packages/'); #sys.path.append('/usr/lib/python3/dist-packages/')
from os import environ
if SIM:
    #environ['ROS_MASTER_URI'] = "http://localhost:1137{}/".format(VOTRE_NUMERO_EQUIPE)
    environ['ROS_MASTER_URI'] = "http://localhost:11311/" #Ordi personnel
else:
    environ['ROS_MASTER_URI'] = "http://192.168.0.53:11311" #cpr-ets05-03:11311/"
    environ['ROS_IP'] = "192.168.0.175"
import rospy
from std_msgs.msg import Float32MultiArray
from jackal_msgs.msg import Drive
from nav_msgs.msg import Odometry
from geometry_msgs.msg import Pose, PoseStamped, PoseWithCovarianceStamped
from sensor_msgs.msg import Imu
from sensor_msgs.msg import CompressedImage
from apriltag_ros.msg import AprilTagDetectionArray

In [ ]:
# Créer et démarrer un nouveau noeud
# (en) Create and start a new node
rospy.init_node('dingo_controller', anonymous=True)

## Création des 'callbacks' (Create callbacks)

Chaque fonction est associée à un sujet ROS et enregistre en continue sont contenu dans une variable globale.

(en) Each function is associated with a ROS topic and continuously records its content in a global variable.

In [ ]:
# Odometry subscriber callback
odom_msg = Pose()
def odom_callback(msg):
    global odom_msg
    odom_msg  = msg.pose.pose

In [ ]:
# Realsense Pose subscriber callback
# pose_msg = Pose()
# def pose_callback(msg):
#     global pose_msg
#     pose_msg = msg.pose.pose
    # pose = PoseStamped()

    # pose.header.seq = 1
    # pose.header.stamp = rospy.Time.now()
    # pose.header.frame_id = "map"

    # pose.pose.position.x = pose_msg.position.x
    # pose.pose.position.y = pose_msg.position.y
    # pose.pose.position.z = 0.0

    # pose.pose.orientation.x = pose_msg.orientation.x
    # pose.pose.orientation.y = pose_msg.orientation.y
    # pose.pose.orientation.z = pose_msg.orientation.z
    # pose.pose.orientation.w = pose_msg.orientation.w

    # pose_pub.publish(pose)

In [ ]:
# AMCL_pose callback (pour la visualisation de la pose)
pose_msg = Pose()
def amcl_callback(msg):
    global pose_msg
    pose_msg = msg.pose.pose
    pose = PoseStamped()

    pose.header.seq = 1
    pose.header.stamp = rospy.Time.now()
    pose.header.frame_id = "map"

    pose.pose.position.x = pose_msg.position.x
    pose.pose.position.y = pose_msg.position.y
    pose.pose.position.z = 0.0

    pose.pose.orientation.x = pose_msg.orientation.x
    pose.pose.orientation.y = pose_msg.orientation.y
    pose.pose.orientation.z = pose_msg.orientation.z
    pose.pose.orientation.w = pose_msg.orientation.w

    pose_pub.publish(pose)

In [ ]:
# AprilTag subscriber callback
tag_array = AprilTagDetectionArray()
def tag_callback(msg):
    global tag_array
    tag_array  = msg

In [ ]:
# Imu subscriber callback
imu_msg = Imu()
def imu_callback(msg):
    global imu_msg
    imu_msg = msg

In [ ]:
# Camera subscriber callback
cam_msg = CompressedImage()
def cam_callback(msg):
    global cam_msg
    cam_msg = msg

In [ ]:
# ROS subscribers et publishers
cmd_drive_pub = rospy.Publisher('/mobile_manip/dingo_velocity_controller/cmd_drive', Drive, queue_size=1)
pose_pub = rospy.Publisher('/mobile_manip/pose', PoseStamped, queue_size=1)
amcl_sub = rospy.Subscriber('/amcl_pose', PoseWithCovarianceStamped, amcl_callback)
#pose_sub = rospy.Subscriber('/mobile_manip/dingo_velocity_controller/odom', Odometry, odom_callback)
#pose_sub = rospy.Subscriber('/mobile_manip/t265/odom/sample', Odometry, pose_callback)
imu_sub = rospy.Subscriber('/imu/data', Imu, imu_callback)
cam_sub = rospy.Subscriber('/mobile_manip/d435i/color/image_raw/compressed', CompressedImage, cam_callback)
tag_sub = rospy.Subscriber('/tag_detections', AprilTagDetectionArray, tag_callback)

## Contrôleur du mouvement (Movement controller)

In [ ]:
interwheel_distance = 0.46
left_wheel_radius = 0.045
right_wheel_radius = 0.045
def move_robot(linear, angular):
    vel_left  = (linear - angular * interwheel_distance / 2.0) / left_wheel_radius
    vel_right = (linear + angular * interwheel_distance / 2.0) / right_wheel_radius

    # Envoi des commandes au roues par topic ROS
    # (en) Sending commands to the wheels via ROS topic
    cmd_drive_msg = Drive()
    cmd_drive_msg.drivers[0] = vel_left
    cmd_drive_msg.drivers[1] = vel_right
    cmd_drive_pub.publish(cmd_drive_msg)

## Interface manuelle (Manual interface)

Permet le contrôle du robot avec des flèches et un retour visuel de la caméra frontale.

(en) Allows the control of the robot with arrows and a visual feedback from the front camera.

In [ ]:
from ipywidgets import HBox, Box, Layout, GridspecLayout
from jupyros import ros3d
import ipywidgets as widgets
import threading
import time
# Image update done with threading, maybe look at ipywebrtc for upgrade

# JUPYTER-ROS
# v = ros3d.Viewer()
# rc = ros3d.ROSConnection(url="ws://localhost:9090")
# tf_client = ros3d.TFClient(ros=rc, fixed_frame='/map')
# g = ros3d.GridModel()
# laser_view = ros3d.LaserScan(topic="/scan", ros=rc, tf_client=tf_client)
# map_view = ros3d.OccupancyGrid(topic="/map", ros=rc, tf_client=tf_client)
# pose_view = ros3d.Pose(topic="/mobile_manip/pose", ros=rc, tf_client=tf_client)
# v.objects = [laser_view, map_view, pose_view]
# v.layout = Layout(border="3px solid black", width="700px", height="600px")

image = widgets.Image(
        value=cam_msg.data,
        format="png",
        height="120%",
        width="200%")

btn_up = widgets.Button(icon='arrow-up')
btn_left = widgets.Button(icon='arrow-left')
btn_down = widgets.Button(icon='arrow-down')
btn_right = widgets.Button(icon='arrow-right')
btn_stop = widgets.Button(description='Stop')

def update_plot():
    while True:
        image.value=cam_msg.data
        time.sleep(0.1)

def on_btn_up_clicked(b):
    move_robot(0.5,0)

def on_btn_left_clicked(b):
    move_robot(0,0.5)

def on_btn_down_clicked(b):
    move_robot(-0.5,0)

def on_btn_right_clicked(b):
    move_robot(0,-0.5)

def on_btn_stop_clicked(b):
    move_robot(0,0)

btn_up.on_click(on_btn_up_clicked)
btn_left.on_click(on_btn_left_clicked)
btn_down.on_click(on_btn_down_clicked)
btn_right.on_click(on_btn_right_clicked)
btn_stop.on_click(on_btn_stop_clicked)

# JUPYTER-ROS
# form = Box(image, layout=Layout(
#     display='flex',
#     flex_flow='column',
#     border='solid 2px',
#     align_items='stretch',
#     width='50%',
#     padding='10px'
# ))

# box = HBox(children=[image, v, btn_up, btn_stop])
# box.layout.padding = "0px"
# box

grid = GridspecLayout(8, 8,height='600px')

grid[0:4,0:3] = image
grid[5,1] = btn_up
grid[6,0] = btn_left
grid[6,1] = btn_stop
grid[6,2] = btn_right
grid[7,1] = btn_down
#grid[:,4:7] = v
thread = threading.Thread(target=update_plot)
thread.start()
grid